# Feature Engineering

As mentioned in the end of the last notebook, we want to enrich our dataset (which currently consists only of single-gameweek data) by adding time-based cumulative features which can provide greater insight regarding player form.

Before we begin, let's look at the features we currently have:

In [1]:
# imports
import pandas as pd

In [2]:
# current features
processed_24_25 = pd.read_csv('../data/processed/2024-25/processed_merged_gws.csv')
current_features = processed_24_25.columns

current_features

Index(['element', 'name', 'position', 'GW', 'total_points', 'value', 'minutes',
       'expected_goals', 'expected_assists', 'expected_goals_conceded',
       'goals_scored', 'assists', 'goals_conceded', 'clean_sheets',
       'ict_index', 'fixture', 'was_home', 'fixture_difficulty'],
      dtype='object')

Now, let's identify the features that we can potentially engineer and enhance:
- **total_points**: this is the player's individual gameweek score. We can enhance this feature by taking the player's total or average gameweek score over the past few weeks.
- We can do the same for **minutes played**, **goals and assists**, **expected goals and assists**, **goals conceded**, **expected goals conceded**, and **clean sheets**.

We will choose a sliding time window of **3 gameweeks** as an initial attempt.

## Features to be added

- **average_score_last_3**:
- **average_minutes_last_3**:
- **goals_last_3**:
- **assists_last_3**:
- **xG_last_3**:
- **xA_last_3**:
- **goals_conceded_last_3**:
- **xGC_last_3**:
- **clean_sheets_last_3**: